In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader
import torch.nn as nn

In [2]:
np.random.seed(42)

In [ ]:
class Datasets:
    def loader_cifar100(batch_size=64, num_workers=2):
        transform = T.Compose([
            T.ToTensor(),
            T.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),  
        ])

        trainset = torchvision.datasets.CIFAR100(root="./data", train=True, download=True, transform=transform)
        testset  = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=transform)

        trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        testloader  = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

        print(f"[CIFAR100] Treino: {len(trainset)} | Teste: {len(testset)}")
        return trainset, testset, trainloader, testloader

    def loader_food101(batch_size=64, num_workers=2):
        transform = T.Compose([
            T.Resize((224, 224)),  
            T.ToTensor(),
            T.Normalize((0.545, 0.436, 0.342), (0.294, 0.275, 0.281))
        ])

        trainset = torchvision.datasets.Food101(root="./data", split="train", download=True, transform=transform)
        testset  = torchvision.datasets.Food101(root="./data", split="test", download=True, transform=transform)

        trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        testloader  = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

        print(f"[Food-101] Treino: {len(trainset)} | Teste: {len(testset)}")
        return trainset, testset, trainloader, testloader

    def loader_caltech256(batch_size=64, num_workers=2):
        transform = T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

        dataset = torchvision.datasets.Caltech256(root="./data", download=True, transform=transform)

        train_size = int(0.8 * len(dataset))
        test_size = len(dataset) - train_size
        trainset, testset = torch.utils.data.random_split(dataset, [train_size, test_size])

        trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        testloader  = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

        print(f"[Caltech-256] Treino: {len(trainset)} | Teste: {len(testset)}")
        return trainset, testset, trainloader, testloader

In [ ]:
# CIFAR-100
cifar_train, cifar_test, cifar_trainloader, cifar_testloader = Datasets.loader_cifar100()

# Food-101
food_train, food_test, food_trainloader, food_testloader = Datasets.loader_food101()

# Caltech-256
caltech_train, caltech_test, caltech_trainloader, caltech_testloader = Datasets.loader_caltech256()

Algoritmo escolhido - VGG: compare sua variante com vgg11, vgg13, vgg16, vgg19 (com ou sem _bn).

protocolo de treino - (épocas, otimizador, LR schedule, augmentation)

Usar VGG como feature extractor e Transformer como classificador:

Blocos de convolução 3×3 empilhados (geralmente 2 ou 3 convs antes de um max pooling).

Camadas totalmente conectadas no final (ou um classificador simples).

Arquitetura “profunda e simples” (sem atalhos ou estruturas complexas).

In [ ]:
import torch
import torch.nn as nn

class VGG_Autoral(nn.Module):
    def __init__(self, num_classes=1000, dropout=0.5):
        super(VGG_Autoral, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(dropout),
            nn.Linear(4096, num_classes),
        )
        
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = VGG_Autoral(num_classes=1000)
#print(model)


VGG_Autoral(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 1

Número de filtros próprio (ex.: 32 → 64 → 128 → 256 → 256)

Batch Normalization após cada conv

Dropout ajustado no classificador

In [4]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # mostra erros exatos da GPU

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import numpy as np
import random
import matplotlib.pyplot as plt
from torchvision.datasets import Food101

# ================================================================
# 🔹 1. Fixar semente para reprodutibilidade
# ================================================================
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# ================================================================
# 🔹 2. Pré-processamento e augmentations
# ================================================================
mean = (0.5071, 0.4867, 0.4408)
std = (0.2675, 0.2565, 0.2761)

train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# ================================================================
# 🔹 3. Dataset e DataLoader (Food-101)
# ================================================================
food_train = Food101(root="./data", split="train", download=True, transform=train_transform)
food_test = Food101(root="./data", split="test", download=True, transform=test_transform)

# Criar uma validação (20%) a partir do treino
train_size = int(0.8 * len(food_train))
val_size = len(food_train) - train_size
train_dataset, val_dataset = random_split(food_train, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(food_test, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

# ================================================================
# 🔹 4. Modelo VGG_Autoral
# ================================================================
class VGG_Autoral(nn.Module):
    def __init__(self, num_classes=101, dropout=0.5):  # <-- corrigido: 101 classes
        super(VGG_Autoral, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 4096), nn.ReLU(True), nn.Dropout(dropout),
            nn.Linear(4096, 4096), nn.ReLU(True), nn.Dropout(dropout),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

# ================================================================
# 🔹 5. Função de treino/validação
# ================================================================
def train_model(model, name, epochs=50):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    best_val, patience, patience_counter = 0, 7, 0
    train_accs, val_accs = [], []

    for epoch in range(epochs):
        # --- Treino ---
        model.train()
        total, correct = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).long()  # <-- garante tipo correto
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

        train_acc = correct / total
        train_accs.append(train_acc)

        # --- Validação ---
        model.eval()
        total, correct = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device).long()
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (preds == labels).sum().item()

        val_acc = correct / total
        val_accs.append(val_acc)
        scheduler.step()

        print(f"[{name}] Época {epoch+1}/{epochs} | Treino: {train_acc:.3f} | Val: {val_acc:.3f}")

        # Early stopping
        if val_acc > best_val:
            best_val = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), f"{name}_best.pth")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"⏹️ Early stopping em {epoch+1}")
                break

    return train_accs, val_accs

# ================================================================
# 🔹 6. Comparação entre variantes
# ================================================================
model_names = {
    "VGG_Autoral": VGG_Autoral(num_classes=101),
    "VGG11_BN": models.vgg11_bn(num_classes=101),
    "VGG13_BN": models.vgg13_bn(num_classes=101),
    "VGG16_BN": models.vgg16_bn(num_classes=101),
    "VGG19_BN": models.vgg19_bn(num_classes=101),
}

results = {}

for name, model in model_names.items():
    print(f"\n🚀 Treinando {name}...")
    model.to(device)
    train_accs, val_accs = train_model(model, name, epochs=50)
    results[name] = (train_accs, val_accs)

# ================================================================
# 🔹 7. Gráfico de acurácia
# ================================================================
plt.figure(figsize=(10, 6))
for name, (train_accs, val_accs) in results.items():
    plt.plot(val_accs, label=f"{name} (val)")
plt.title("Comparação de acurácia de validação - Família VGG (Food-101)")
plt.xlabel("Épocas")
plt.ylabel("Acurácia")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


Usando dispositivo: cuda

🚀 Treinando VGG_Autoral...


c:\Users\bruno\ICA\Inteligencia_computacional_aplicada\venv\Lib\site-packages\torch\cuda\__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5070 Ti with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5070 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


RuntimeError: CUDA error: no kernel image is available for execution on the device
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
import torch
print("CUDA disponível:", torch.cuda.is_available())
print("Versão PyTorch:", torch.__version__)
print("Versão CUDA:", torch.version.cuda)
print("Placa detectada:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Nenhuma")


CUDA disponível: True
Versão PyTorch: 2.6.0+cu124
Versão CUDA: 12.4
Placa detectada: NVIDIA GeForce RTX 5070 Ti


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
from collections import defaultdict
import random
import os

# Configuração de device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando device: {device}")

# Definir seed para reprodutibilidade
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

# Carregar datasets
print("\n" + "="*50)
print("CARREGANDO DATASETS")
print("="*50)
train_loaders, test_loaders = load_datasets(seed=42)

# Ajustar número de classes para cada dataset
num_classes_por_dataset = {
    "Food-101": 101,
    "Tiny-ImageNet": 200,
    "CIFAR-100": 100
}

# Inicializar modelos para cada dataset
def criar_modelos_para_dataset(num_classes):
    """Cria instâncias dos modelos VGG com número correto de classes"""
    vgg11 = models.vgg11(weights=None)
    vgg11.classifier[6] = nn.Linear(4096, num_classes)
    
    vgg13 = models.vgg13(weights=None)
    vgg13.classifier[6] = nn.Linear(4096, num_classes)
    
    vgg16 = models.vgg16(weights=None)
    vgg16.classifier[6] = nn.Linear(4096, num_classes)
    
    vgg19 = models.vgg19(weights=None)
    vgg19.classifier[6] = nn.Linear(4096, num_classes)
    
    vgg_autoral = VGG_Autoral(num_classes=num_classes)
    
    return {
        "VGG_Autoral": vgg_autoral,
        "VGG11": vgg11,
        "VGG13": vgg13,
        "VGG16": vgg16,
        "VGG19": vgg19
    }

# Dicionário para armazenar todos os resultados
resultados_completos = {}
modelos_treinados = {}

# Loop principal: treinar todos os modelos em todos os datasets
print("\n" + "="*50)
print("INICIANDO TREINAMENTO")
print("="*50)

for dataset_name in ["Food-101", "Tiny-ImageNet", "CIFAR-100"]:
    print(f"\n{'='*50}")
    print(f"DATASET: {dataset_name}")
    print(f"{'='*50}")
    
    num_classes = num_classes_por_dataset[dataset_name]
    modelos = criar_modelos_para_dataset(num_classes)
    
    train_loader = train_loaders[dataset_name]
    test_loader = test_loaders[dataset_name]
    
    # Criar validation loader (20% do test set)
    test_dataset = test_loader.dataset
    val_size = int(0.2 * len(test_dataset))
    test_size = len(test_dataset) - val_size
    val_dataset, _ = torch.utils.data.random_split(
        test_dataset, 
        [val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)
    
    modelos_treinados[dataset_name] = {}
    
    for nome_modelo, modelo in modelos.items():
        print(f"\n{'-'*50}")
        print(f"Treinando {nome_modelo} em {dataset_name}")
        print(f"{'-'*50}")
        
        # Treinar modelo
        modelo_treinado, historico = train_model(
            modelo, 
            train_loader, 
            val_loader, 
            device=device,
            optimizer_name="AdamW",
            lr=0.001,
            scheduler_type="cosine",
            epochs=50,
            patience=10
        )
        
        # Salvar modelo treinado
        modelos_treinados[dataset_name][nome_modelo] = modelo_treinado
        
        # Avaliar no test set
        print(f"\nAvaliando {nome_modelo} no test set...")
        metricas = Metrics.evaluate_model(modelo_treinado, test_loader, device)
        
        # Obter complexidade e desempenho
        complexidade = Metrics.get_model_complexity(modelo_treinado)
        desempenho = Metrics.benchmark_model(modelo_treinado, device)
        
        # Armazenar resultados
        chave = f"{dataset_name}_{nome_modelo}"
        resultados_completos[chave] = {
            **metricas,
            **complexidade,
            **desempenho,
            "historico": historico
        }
        
        print(f"\nResultados {nome_modelo}:")
        print(f"  Acurácia: {metricas['Acurácia']:.4f}")
        print(f"  F1-Score: {metricas['F1']:.4f}")
        print(f"  NLL: {metricas['NLL']:.4f}")
        print(f"  ECE: {metricas['ECE']:.4f}")
        print(f"  Parâmetros: {complexidade['Parâmetros']/1e6:.2f}M")
        print(f"  Latência: {desempenho['Latência (s)']:.4f}s")

print("\n" + "="*50)
print("TREINAMENTO CONCLUÍDO")
print("="*50)

# GERAR GRÁFICOS E ANÁLISES
print("\n" + "="*50)
print("GERANDO GRÁFICOS E ANÁLISES")
print("="*50)

# 1. Comparação de Acurácia por Dataset
plt.figure(figsize=(14, 6))
for i, dataset_name in enumerate(["Food-101", "Tiny-ImageNet", "CIFAR-100"], 1):
    plt.subplot(1, 3, i)
    nomes = []
    acuracias = []
    for nome_modelo in ["VGG_Autoral", "VGG11", "VGG13", "VGG16", "VGG19"]:
        chave = f"{dataset_name}_{nome_modelo}"
        nomes.append(nome_modelo)
        acuracias.append(resultados_completos[chave]["Acurácia"])
    
    cores = ['#d62828', '#f77f00', '#fcbf49', '#06d6a0', '#118ab2']
    plt.bar(nomes, acuracias, color=cores)
    plt.title(f"{dataset_name}", fontsize=12, fontweight='bold')
    plt.ylabel("Acurácia")
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.3)
    plt.ylim([0, 1])

plt.tight_layout()
plt.savefig('comparacao_acuracia.png', dpi=300, bbox_inches='tight')
plt.show()

# 2. Comparação de Complexidade (Parâmetros vs FLOPs)
plt.figure(figsize=(10, 6))
for dataset_name in ["Food-101", "Tiny-ImageNet", "CIFAR-100"]:
    params = []
    flops = []
    nomes = []
    for nome_modelo in ["VGG_Autoral", "VGG11", "VGG13", "VGG16", "VGG19"]:
        chave = f"{dataset_name}_{nome_modelo}"
        params.append(resultados_completos[chave]["Parâmetros"] / 1e6)
        flops.append(resultados_completos[chave]["FLOPs"] / 1e9)
        nomes.append(nome_modelo)
    
    plt.scatter(params, flops, label=dataset_name, s=100, alpha=0.7)
    
    for i, nome in enumerate(nomes):
        if dataset_name == "Food-101":
            plt.annotate(nome, (params[i], flops[i]), 
                        xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.xlabel("Parâmetros (Milhões)")
plt.ylabel("FLOPs (GFLOPs)")
plt.title("Complexidade dos Modelos")
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('complexidade_modelos.png', dpi=300, bbox_inches='tight')
plt.show()

# 3. Comparação de Métricas (Precision, Recall, F1)
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
metricas_plot = ["Precisão", "Recall", "F1"]

for idx, metrica in enumerate(metricas_plot):
    ax = axes[idx]
    x = np.arange(5)
    width = 0.25
    
    for i, dataset_name in enumerate(["Food-101", "Tiny-ImageNet", "CIFAR-100"]):
        valores = []
        for nome_modelo in ["VGG_Autoral", "VGG11", "VGG13", "VGG16", "VGG19"]:
            chave = f"{dataset_name}_{nome_modelo}"
            valores.append(resultados_completos[chave][metrica])
        
        ax.bar(x + i*width, valores, width, label=dataset_name)
    
    ax.set_xlabel("Modelo")
    ax.set_ylabel(metrica)
    ax.set_title(f"Comparação de {metrica}")
    ax.set_xticks(x + width)
    ax.set_xticklabels(["VGG_Autoral", "VGG11", "VGG13", "VGG16", "VGG19"], rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig('comparacao_metricas.png', dpi=300, bbox_inches='tight')
plt.show()

# 4. Calibração (NLL e ECE)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, metrica in enumerate(["NLL", "ECE"]):
    ax = axes[idx]
    x = np.arange(5)
    width = 0.25
    
    for i, dataset_name in enumerate(["Food-101", "Tiny-ImageNet", "CIFAR-100"]):
        valores = []
        for nome_modelo in ["VGG_Autoral", "VGG11", "VGG13", "VGG16", "VGG19"]:
            chave = f"{dataset_name}_{nome_modelo}"
            valores.append(resultados_completos[chave][metrica])
        
        ax.bar(x + i*width, valores, width, label=dataset_name)
    
    ax.set_xlabel("Modelo")
    ax.set_ylabel(metrica)
    ax.set_title(f"Calibração - {metrica}")
    ax.set_xticks(x + width)
    ax.set_xticklabels(["VGG_Autoral", "VGG11", "VGG13", "VGG16", "VGG19"], rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig('calibracao_modelos.png', dpi=300, bbox_inches='tight')
plt.show()

# 5. Latência de Inferência
plt.figure(figsize=(12, 6))
x = np.arange(5)
width = 0.25

for i, dataset_name in enumerate(["Food-101", "Tiny-ImageNet", "CIFAR-100"]):
    latencias = []
    for nome_modelo in ["VGG_Autoral", "VGG11", "VGG13", "VGG16", "VGG19"]:
        chave = f"{dataset_name}_{nome_modelo}"
        latencias.append(resultados_completos[chave]["Latência (s)"] * 1000)  # Converter para ms
    
    plt.bar(x + i*width, latencias, width, label=dataset_name)

plt.xlabel("Modelo")
plt.ylabel("Latência (ms)")
plt.title("Latência de Inferência")
plt.xticks(x + width, ["VGG_Autoral", "VGG11", "VGG13", "VGG16", "VGG19"], rotation=45, ha='right')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.savefig('latencia_inferencia.png', dpi=300, bbox_inches='tight')
plt.show()

# 6. Curvas de Aprendizado (Loss e Acurácia)
for dataset_name in ["Food-101", "Tiny-ImageNet", "CIFAR-100"]:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for nome_modelo in ["VGG_Autoral", "VGG11", "VGG13", "VGG16", "VGG19"]:
        chave = f"{dataset_name}_{nome_modelo}"
        hist = resultados_completos[chave]["historico"]
        
        # Loss
        axes[0].plot(hist["train_loss"], label=f"{nome_modelo} (Train)", linestyle='--', alpha=0.7)
        axes[0].plot(hist["val_loss"], label=f"{nome_modelo} (Val)")
        
        # Acurácia
        axes[1].plot(hist["train_acc"], label=f"{nome_modelo} (Train)", linestyle='--', alpha=0.7)
        axes[1].plot(hist["val_acc"], label=f"{nome_modelo} (Val)")
    
    axes[0].set_xlabel("Época")
    axes[0].set_ylabel("Loss")
    axes[0].set_title(f"{dataset_name} - Curva de Loss")
    axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    axes[0].grid(True, alpha=0.3)
    
    axes[1].set_xlabel("Época")
    axes[1].set_ylabel("Acurácia")
    axes[1].set_title(f"{dataset_name} - Curva de Acurácia")
    axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'curvas_aprendizado_{dataset_name.replace("-", "_")}.png', dpi=300, bbox_inches='tight')
    plt.show()

# 7. Tabela Resumo
print("\n" + "="*80)
print("TABELA RESUMO DE RESULTADOS")
print("="*80)
print(f